In [ ]:
# ETL con Polars en Google Colab
# Instalar Polars (ejecutar solo una vez)
# !pip install polars

import polars as pl
from datetime import datetime, timedelta
import random

In [ ]:
# =============================================================================
# EXTRACT - Extracción de datos
# =============================================================================

print("=" * 60)
print("FASE 1: EXTRACT - Extracción de datos")
print("=" * 60)

# Simular datos de ventas (en un caso real, leerías desde CSV, API, DB, etc.)
n_records = 1000
start_date = datetime(2024, 1, 1)

data = {
    "id_venta": range(1, n_records + 1),
    "fecha": [(start_date + timedelta(days=random.randint(0, 365))).strftime("%Y-%m-%d")
              for _ in range(n_records)],
    "producto": [random.choice(["Laptop", "Mouse", "Teclado", "Monitor", "Webcam"])
                 for _ in range(n_records)],
    "cantidad": [random.randint(1, 10) for _ in range(n_records)],
    "precio_unitario": [round(random.uniform(10, 1000), 2) for _ in range(n_records)],
    "cliente": [f"Cliente_{random.randint(1, 100)}" for _ in range(n_records)],
    "region": [random.choice(["Norte", "Sur", "Este", "Oeste", None]) for _ in range(n_records)],
    "estado": [random.choice(["Completado", "Pendiente", "Cancelado"]) for _ in range(n_records)]
}

# Crear DataFrame de Polars
df_raw = pl.DataFrame(data)

print(f"\n✓ Datos extraídos: {df_raw.shape[0]} registros, {df_raw.shape[1]} columnas")
print("\nPrimeras 5 filas:")
print(df_raw.head())
print("\nInformación del DataFrame:")
print(df_raw.describe())

In [ ]:
# =============================================================================
# TRANSFORM - Transformación de datos
# =============================================================================

print("\n" + "=" * 60)
print("FASE 2: TRANSFORM - Transformación de datos")
print("=" * 60)

# 1. Convertir fecha a tipo datetime
df_transformed = df_raw.with_columns([
    pl.col("fecha").str.to_date().alias("fecha")
])

# 2. Crear columna de total de venta
df_transformed = df_transformed.with_columns([
    (pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta")
])

# 3. Filtrar solo ventas completadas
df_transformed = df_transformed.filter(pl.col("estado") == "Completado")

# 4. Manejar valores nulos en región (rellenar con "No Especificado")
df_transformed = df_transformed.with_columns([
    pl.col("region").fill_null("No Especificado")
])

# 5. Extraer información temporal
df_transformed = df_transformed.with_columns([
    pl.col("fecha").dt.year().alias("año"),
    pl.col("fecha").dt.month().alias("mes"),
    pl.col("fecha").dt.quarter().alias("trimestre")
])

# 6. Categorizar ventas por monto
df_transformed = df_transformed.with_columns([
    pl.when(pl.col("total_venta") < 100)
    .then(pl.lit("Baja"))
    .when(pl.col("total_venta") < 500)
    .then(pl.lit("Media"))
    .otherwise(pl.lit("Alta"))
    .alias("categoria_venta")
])

print(f"\n✓ Datos transformados: {df_transformed.shape[0]} registros")
print("\nPrimeras 5 filas transformadas:")
print(df_transformed.head())


In [ ]:
# =============================================================================
# AGREGACIONES Y ANÁLISIS
# =============================================================================

print("\n" + "=" * 60)
print("AGREGACIONES Y ANÁLISIS")
print("=" * 60)

# Ventas por producto
ventas_producto = (
    df_transformed
    .group_by("producto")
    .agg([
        pl.count().alias("num_ventas"),
        pl.sum("cantidad").alias("cantidad_total"),
        pl.sum("total_venta").alias("ingreso_total"),
        pl.mean("total_venta").alias("ticket_promedio")
    ])
    .sort("ingreso_total", descending=True)
)

print("\n📊 Ventas por Producto:")
print(ventas_producto)

# Ventas por región y mes
ventas_region_mes = (
    df_transformed
    .group_by(["region", "mes"])
    .agg([
        pl.count().alias("num_ventas"),
        pl.sum("total_venta").alias("ingreso_total")
    ])
    .sort(["region", "mes"])
)

print("\n📊 Ventas por Región y Mes (primeras 10):")
print(ventas_region_mes.head(10))


In [ ]:
# =============================================================================
# LOAD - Carga de datos
# =============================================================================

print("\n" + "=" * 60)
print("FASE 3: LOAD - Carga de datos")
print("=" * 60)

# Guardar datos transformados
df_transformed.write_csv("ventas_transformadas.csv")
print("✓ Archivo CSV guardado: ventas_transformadas.csv")

# Guardar agregaciones
ventas_producto.write_csv("ventas_por_producto.csv")
print("✓ Archivo CSV guardado: ventas_por_producto.csv")

ventas_region_mes.write_csv("ventas_por_region_mes.csv")
print("✓ Archivo CSV guardado: ventas_por_region_mes.csv")

# Guardar en formato Parquet (más eficiente)
df_transformed.write_parquet("ventas_transformadas.parquet")

In [ ]:
# =============================================================================
# RESUMEN DEL ETL
# =============================================================================

print("\n" + "=" * 60)
print("RESUMEN DEL ETL")
print("=" * 60)

print(f"""
📈 Estadísticas Finales:
  - Registros originales: {df_raw.shape[0]}
  - Registros procesados: {df_transformed.shape[0]}
  - Registros descartados: {df_raw.shape[0] - df_transformed.shape[0]}
  - Columnas finales: {df_transformed.shape[1]}
  - Ingreso total: ${df_transformed['total_venta'].sum():,.2f}
  - Ticket promedio: ${df_transformed['total_venta'].mean():,.2f}
  - Período analizado: {df_transformed['fecha'].min()} a {df_transformed['fecha'].max()}
""")

print("\n✅ ETL completado exitosamente!")